In [45]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [46]:
# Load the dataset
df = pd.read_csv(r'D:\uni\SoftwareEngineering\group project\GroupProject\project_dir\machine_learning\combined_data.csv')
print(df.dtypes)

name                object
number               int64
datequery           object
dayquery            object
hourquery            int64
available_bikes      int64
temperature        float64
wind_speed         float64
pressure             int64
humidity             int64
weather_main        object
dtype: object


In [51]:
# Perform data preprocessing
# Convert datequery and dayquery columns to datetime type
df['datequery'] = pd.to_datetime(df['datequery'])
#df['dayquery'] = pd.to_datetime(df['dayquery']).day_name()

# Drop the number column if it's not relevant to your analysis
df.drop('number', axis=1, inplace=True)

# Perform one-hot encoding on categorical features
# Check for problems
categorical_features = ['name', 'dayquery', 'weather_main']
for feature in categorical_features:
    encoded_feature = pd.get_dummies(df[feature], prefix=feature)
    df = pd.concat([df, encoded_feature], axis=1)
    df.drop([feature], axis=1, inplace=True)

# Normalize continuous features
continuous_features = ['number', 'hourquery', 'available_bikes', 'temperature', 'wind_speed', 'pressure', 'humidity']
scaler = StandardScaler()
df[continuous_features] = scaler.fit_transform(df[continuous_features])

# Create a new DataFrame by concatenating one-hot encoded and normalized continuous features
new_df = pd.concat([df.filter(regex='^name_|^weather_main_'),
                   df[continuous_features]], axis=1)

# Define input features and target variable
input_features = new_df.columns.tolist()
input_features.remove('available_bikes')
target_variable = 'available_bikes'


KeyError: "['number'] not found in axis"

In [49]:
# Split the dataset into train and test sets using time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(df):
    train_df = df.loc[train_index]
    test_df = df.loc[test_index]

# Prepare the input features and target variable for training and testing
X_train, y_train = train_df[input_features], train_df[target_variable]
X_test, y_test = test_df[input_features], test_df[target_variable]

In [50]:
# Train a Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model using RMSE
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Test RMSE: {rmse}')

Test RMSE: 9.604777089207268
